In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/1984.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("빅토리 맨션에 대해서 설명해줘")

AIMessage(content="빅토리 맨션은 윈스턴 스미스가 살고 있는 아파트로, 냄새가 나는 복숭아와 오래된 러그 매트의 냄새가 나는 현관이 있습니다. 아파트에는 실내 전시용으로 너무 큰 컬러 포스터가 벽에 박혀 있으며, 약 45세의 남자의 거대한 얼굴이 그려져 있습니다. 아파트는 엘리베이터를 시도해봐도 소용이 없을 정도로 가끔 작동하지 않고, 현재는 적대주의 주간을 위한 경제 절약을 위해 주간 중에는 전기가 차단되어 있습니다. 아파트는 7층으로 윈스턴은 39세이며 오른쪽 발목 위에 정맥류가 있어 천천히 올라가며 여러 차례 쉬면서 올라갑니다. 각 층마다 엘리베이터 샤프트 맞은편에 걸린 거대한 얼굴이 그려진 포스터가 벽에 걸려 있습니다. 이 그림은 당신이 움직일 때 눈이 따라다니는 그림 중 하나입니다. 아래에는 '빅 브라더가 당신을 지켜보고 있다'는 캡션이 적혀 있습니다.")